In [1]:
import os, sys, os, threading, re

# 1) Tell TF to hide INFO logs, but keep WARNING+ERROR
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 2) Build a small stderr filter that drops only known spam lines
patterns = [
    r'numa_node',                             # NUMA spam
    r'Unable to register cuFFT factory',      # duplicate cuFFT
    r'Unable to register cuDNN factory',      # duplicate cuDNN
    r'Unable to register cuBLAS factory',     # duplicate cuBLAS
    r'gpu_timer\.cc:114',                     # "Skipping the delay kernel"
    r"\+ptx85' is not a recognized feature",  # PTX feature noise
    r"Your kernel may have been built without NUMA support.",  # PTX feature noise
]

compiled = [re.compile(p) for p in patterns]

# We’ll redirect C++ stderr (fd 2) into a pipe, then read/filter in Python
r_fd, w_fd = os.pipe()
orig_stderr_fd = os.dup(2)        # save real stderr

# Point fd 2 (stderr) to the write end of our pipe
os.dup2(w_fd, 2)

def stderr_filter_thread():
    # read from pipe (r_fd), write filtered lines to original stderr (orig_stderr_fd)
    with os.fdopen(r_fd, 'r') as stream, os.fdopen(orig_stderr_fd, 'w') as real_stderr:
        for line in stream:
            if any(p.search(line) for p in compiled):
                # drop known-noise lines
                continue
            real_stderr.write(line)
            real_stderr.flush()

threading.Thread(target=stderr_filter_thread, daemon=True).start()

print("TF log filter installed: INFO hidden, noisy C++ spam filtered, real warnings/errors will still show.")


TF log filter installed: INFO hidden, noisy C++ spam filtered, real warnings/errors will still show.


In [2]:
import tensorflow as tf
tf.config.optimizer.set_jit(False)
import numpy as np

In [3]:
print(tf.__version__)

2.17.0


In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
scalar = tf.constant(7)
scalar, scalar.shape, scalar.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=7>, TensorShape([]), 0)

In [9]:
vector = tf.constant([3, 5, 7, 10])
vector, vector.shape, vector.ndim

(<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 3,  5,  7, 10], dtype=int32)>,
 TensorShape([4]),
 1)

In [10]:
matrix = tf.constant([[3, 5, 7, 10],
                     [1, 2, 3, 4]])
matrix, matrix.shape, matrix.ndim

(<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
 array([[ 3,  5,  7, 10],
        [ 1,  2,  3,  4]], dtype=int32)>,
 TensorShape([2, 4]),
 2)

In [13]:
tensor = tf.random.uniform(shape=[3, 6, 7], minval=1, maxval=10, dtype=tf.float32)
tensor, tensor.shape, tensor.ndim

(<tf.Tensor: shape=(3, 6, 7), dtype=float32, numpy=
 array([[[3.0941572, 9.137987 , 1.1435164, 8.409265 , 4.8483787,
          3.2855372, 7.757273 ],
         [3.6393194, 7.153599 , 2.412107 , 3.1040106, 6.504174 ,
          3.562282 , 3.1896546],
         [7.351572 , 2.931466 , 6.561021 , 9.442308 , 6.7046404,
          7.3704963, 1.9543246],
         [9.299803 , 6.4541507, 9.683387 , 4.2120376, 7.9961557,
          5.4483943, 5.228716 ],
         [1.3750072, 2.3244653, 2.8837185, 7.877821 , 8.85671  ,
          2.0707603, 9.020347 ],
         [2.957859 , 2.225761 , 9.554733 , 5.4206924, 8.814623 ,
          6.92774  , 2.9279675]],
 
        [[7.029641 , 5.1233063, 8.969457 , 9.297218 , 1.4241002,
          6.1015744, 3.7416887],
         [1.9443017, 3.1972163, 3.2110276, 6.9973087, 4.4741817,
          6.0952144, 9.929061 ],
         [2.9293656, 5.991886 , 7.4525228, 7.3722916, 4.0240417,
          1.0969082, 1.5056221],
         [2.4504657, 9.489074 , 9.94546  , 3.6666157, 6.5616107

In [16]:
seeded_generator_1 = tf.random.Generator.from_seed(42)
random_a = seeded_generator_1.normal(shape=(3, 2))

seeded_generator_2 = tf.random.Generator.from_seed(42)
random_b = seeded_generator_2.normal(shape=(3, 2))

random_a == random_b

<tf.Tensor: shape=(3, 2), dtype=bool, numpy=
array([[ True,  True],
       [ True,  True],
       [ True,  True]])>

In [18]:
changeable_tensor = tf.Variable([3., 5.])
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([3., 5.], dtype=float32)>

In [20]:
changeable_tensor[0].assign(-5.)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float32, numpy=array([-5.,  5.], dtype=float32)>

In [21]:
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([-5.,  5.], dtype=float32)>

In [24]:
tensor = tf.random.uniform(minval=1, maxval=20, shape=(3, 8), dtype=tf.int32)
tensor

<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
array([[ 7,  3, 12,  6, 14, 19, 17,  1],
       [ 1,  8, 14, 11,  9, 19,  5,  6],
       [12,  2, 14, 16,  2,  5,  2, 19]], dtype=int32)>

In [27]:
tensor_shuffled = tf.random.shuffle(tensor, seed=42)
tensor_shuffled

<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
array([[12,  2, 14, 16,  2,  5,  2, 19],
       [ 1,  8, 14, 11,  9, 19,  5,  6],
       [ 7,  3, 12,  6, 14, 19, 17,  1]], dtype=int32)>

In [29]:
tensor = tf.constant(np.random.randint(1, 10, size=(3, 5, 2)))
tensor

<tf.Tensor: shape=(3, 5, 2), dtype=int64, numpy=
array([[[7, 7],
        [3, 4],
        [6, 5],
        [1, 8],
        [4, 6]],

       [[3, 2],
        [8, 6],
        [9, 7],
        [2, 2],
        [7, 6]],

       [[4, 7],
        [3, 6],
        [5, 2],
        [2, 7],
        [1, 6]]])>

In [31]:
tf.ones(shape=(3, 2, 5))

<tf.Tensor: shape=(3, 2, 5), dtype=float32, numpy=
array([[[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]], dtype=float32)>

In [32]:
tf.zeros(shape=(2, 3, 5))

<tf.Tensor: shape=(2, 3, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [34]:
tf.constant(np.random.randint(1, 100, size=(2, 3, 5)), dtype=tf.float32)

<tf.Tensor: shape=(2, 3, 5), dtype=float32, numpy=
array([[[75., 37., 78., 14., 10.],
        [95., 99., 86., 19., 40.],
        [12., 26.,  3., 18., 60.]],

       [[50., 41., 28., 38., 38.],
        [87., 36., 28., 10., 86.],
        [22., 22.,  7., 98., 65.]]], dtype=float32)>

In [36]:
tf.constant(np.random.normal(size=(2, 3, 5)), dtype=tf.float32)

<tf.Tensor: shape=(2, 3, 5), dtype=float32, numpy=
array([[[-2.1437502 ,  0.30921546, -0.22185636, -1.3934776 ,
          1.4139794 ],
        [-0.56961596, -0.4945409 , -0.11157782, -1.1236547 ,
         -0.3942583 ],
        [ 0.50905234,  1.1088498 ,  0.6358182 , -0.10864833,
         -0.8342382 ]],

       [[ 0.54122484, -0.8083795 ,  0.25335148, -1.3346981 ,
          0.79379183],
        [-0.87532616, -1.8738736 ,  0.6031991 ,  0.27659944,
         -0.4343053 ],
        [ 0.08919819,  0.15727723, -2.0088034 , -1.0384914 ,
          0.266972  ]]], dtype=float32)>

In [40]:
np.arange(1, 10, dtype=np.int32, step=2)

array([1, 3, 5, 7, 9], dtype=int32)

In [43]:
tf.constant(np.arange(1, 10, dtype=np.int32, step=2))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 3, 5, 7, 9], dtype=int32)>

In [44]:
numpy_a = np.arange(1, 25)
tf_a = tf.constant(numpy_a, shape=(2, 6, 2))
tf_b = tf.constant(numpy_a, shape=(2, 12))
tf_c = tf.constant(numpy_a)
tf_a, tf_b, tf_c

(<tf.Tensor: shape=(2, 6, 2), dtype=int64, numpy=
 array([[[ 1,  2],
         [ 3,  4],
         [ 5,  6],
         [ 7,  8],
         [ 9, 10],
         [11, 12]],
 
        [[13, 14],
         [15, 16],
         [17, 18],
         [19, 20],
         [21, 22],
         [23, 24]]])>,
 <tf.Tensor: shape=(2, 12), dtype=int64, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12],
        [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]])>,
 <tf.Tensor: shape=(24,), dtype=int64, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])>)

In [45]:
tf_a.shape, tf_b.ndim, tf.size(tf_b[0])

(TensorShape([2, 6, 2]), 2, <tf.Tensor: shape=(), dtype=int32, numpy=12>)

In [46]:
tf_a[:1, :2, :]

<tf.Tensor: shape=(1, 2, 2), dtype=int64, numpy=
array([[[1, 2],
        [3, 4]]])>

In [48]:
ran_3_tf = tf_a[..., tf.newaxis]
ran_3_tf

<tf.Tensor: shape=(2, 6, 2, 1), dtype=int64, numpy=
array([[[[ 1],
         [ 2]],

        [[ 3],
         [ 4]],

        [[ 5],
         [ 6]],

        [[ 7],
         [ 8]],

        [[ 9],
         [10]],

        [[11],
         [12]]],


       [[[13],
         [14]],

        [[15],
         [16]],

        [[17],
         [18]],

        [[19],
         [20]],

        [[21],
         [22]],

        [[23],
         [24]]]])>

In [54]:
tf.expand_dims(ran_3_tf, axis=0)

<tf.Tensor: shape=(1, 2, 6, 2, 1), dtype=int64, numpy=
array([[[[[ 1],
          [ 2]],

         [[ 3],
          [ 4]],

         [[ 5],
          [ 6]],

         [[ 7],
          [ 8]],

         [[ 9],
          [10]],

         [[11],
          [12]]],


        [[[13],
          [14]],

         [[15],
          [16]],

         [[17],
          [18]],

         [[19],
          [20]],

         [[21],
          [22]],

         [[23],
          [24]]]]])>

In [75]:
np.random.seed(42)
tensor = tf.constant(np.random.ranf(size=(2, 3)), dtype=tf.float32)
tensor

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.37454012, 0.9507143 , 0.7319939 ],
       [0.5986585 , 0.15601864, 0.15599452]], dtype=float32)>

In [61]:
tf.add(tensor, 1.0)

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[1.37454012, 1.95071431, 1.73199394],
       [1.59865848, 1.15601864, 1.15599452]])>

In [64]:
tensor

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[0.37454012, 0.95071431, 0.73199394],
       [0.59865848, 0.15601864, 0.15599452]])>

In [65]:
tf.subtract(tensor, 1.0)

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[-0.62545988, -0.04928569, -0.26800606],
       [-0.40134152, -0.84398136, -0.84400548]])>

In [66]:
tensor + 1

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[1.37454012, 1.95071431, 1.73199394],
       [1.59865848, 1.15601864, 1.15599452]])>

In [67]:
tensor - 1

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[-0.62545988, -0.04928569, -0.26800606],
       [-0.40134152, -0.84398136, -0.84400548]])>

In [68]:
tf.multiply(tensor, 5)

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[1.87270059, 4.75357153, 3.65996971],
       [2.99329242, 0.7800932 , 0.7799726 ]])>

In [69]:
tf.divide(tensor, 5)

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[0.07490802, 0.19014286, 0.14639879],
       [0.1197317 , 0.03120373, 0.0311989 ]])>

In [70]:
tensor

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[0.37454012, 0.95071431, 0.73199394],
       [0.59865848, 0.15601864, 0.15599452]])>

In [73]:
tf_2 = tf.constant(np.random.randint(1, 10, size=(3, 2)), dtype=tf.float32)
tf_2

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[6., 5.],
       [2., 8.],
       [6., 2.]], dtype=float32)>

In [76]:
tensor @ tf_2

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 8.540632 , 10.942402 ],
       [ 4.8399553,  4.5534306]], dtype=float32)>

In [77]:
tf_2 @ tensor

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[5.2405334, 6.484379 , 5.171936 ],
       [5.538348 , 3.1495776, 2.711944 ],
       [3.4445577, 6.016323 , 4.7039523]], dtype=float32)>

In [78]:
tf.matmul(tensor, tf_2)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 8.540632 , 10.942402 ],
       [ 4.8399553,  4.5534306]], dtype=float32)>

In [79]:
tf.matmul(tf_2, tensor)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[5.2405334, 6.484379 , 5.171936 ],
       [5.538348 , 3.1495776, 2.711944 ],
       [3.4445577, 6.016323 , 4.7039523]], dtype=float32)>

In [80]:
tensor.shape, tf_2.shape

(TensorShape([2, 3]), TensorShape([3, 2]))

In [83]:
tf_3 = tf.constant(np.random.ranf(size=(3, 1)), dtype=tf.float32)
tf_3

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.7080726 ],
       [0.02058449],
       [0.96990985]], dtype=float32)>

In [84]:
tf.matmul(tensor, tf_3)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.99473965],
       [0.57840586]], dtype=float32)>

In [85]:
tf_4 = tf.reshape(tf_3, shape=(1, 3))

In [87]:
tf.matmul(tf_4, tf.reshape(tensor, shape=(3, 2)))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.43159336, 0.8367984 ]], dtype=float32)>

In [89]:
tf.transpose(tensor), tensor

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[0.37454012, 0.5986585 ],
        [0.9507143 , 0.15601864],
        [0.7319939 , 0.15599452]], dtype=float32)>,
 <tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[0.37454012, 0.9507143 , 0.7319939 ],
        [0.5986585 , 0.15601864, 0.15599452]], dtype=float32)>)

In [91]:
tf5 = tf.random.normal(shape=(3, 8))
tf5

<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[-0.7083535 , -0.74329644,  0.0980624 ,  1.329291  ,  0.1857169 ,
        -1.5237479 ,  0.2110184 , -1.0662823 ],
       [-1.5388482 , -1.7848148 ,  0.63565636, -1.2826623 , -0.34186205,
         0.3771491 , -0.1049398 ,  0.83116615],
       [ 0.6645473 , -0.6785432 ,  0.05911025, -0.6456633 , -1.1344243 ,
        -0.6532986 , -0.98629934, -0.19512208]], dtype=float32)>

In [93]:
tf6 = tf.constant(np.random.randint(-10, 10, size=(3, 8)), dtype=tf.float32)
tf6

<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[  1.,  -5.,  -9., -10.,   1.,   1.,   6.,  -1.],
       [  5.,   4.,   4.,   8.,   1.,   9.,  -8.,  -6.],
       [  8.,  -4.,  -2.,  -4.,   7.,  -7.,   3.,   7.]], dtype=float32)>

In [94]:
tf.abs(tf6)

<tf.Tensor: shape=(3, 8), dtype=float32, numpy=
array([[ 1.,  5.,  9., 10.,  1.,  1.,  6.,  1.],
       [ 5.,  4.,  4.,  8.,  1.,  9.,  8.,  6.],
       [ 8.,  4.,  2.,  4.,  7.,  7.,  3.,  7.]], dtype=float32)>

In [96]:
tf.reduce_max(tf6)

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [97]:
tf.reduce_min(tf6)

<tf.Tensor: shape=(), dtype=float32, numpy=-10.0>

In [98]:
tf.reduce_mean(tf6)

<tf.Tensor: shape=(), dtype=float32, numpy=0.375>

In [99]:
tf.reduce_sum(tf6)

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [100]:
tf.math.reduce_std(tf6)

<tf.Tensor: shape=(), dtype=float32, numpy=5.7793636>

In [103]:
tf.math.reduce_variance(tf6).numpy(), tf.math.reduce_variance(tf6)

(33.401043, <tf.Tensor: shape=(), dtype=float32, numpy=33.401043>)

In [109]:
tf7 = tf.constant([4, 2, 3, 1, 0, 2, 6])
tf.one_hot(tf7, depth=5)

<tf.Tensor: shape=(7, 5), dtype=float32, numpy=
array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

In [110]:
tf.one_hot(tf7, depth=5, on_value='YES', off_value='NO')

<tf.Tensor: shape=(7, 5), dtype=string, numpy=
array([[b'NO', b'NO', b'NO', b'NO', b'YES'],
       [b'NO', b'NO', b'YES', b'NO', b'NO'],
       [b'NO', b'NO', b'NO', b'YES', b'NO'],
       [b'NO', b'YES', b'NO', b'NO', b'NO'],
       [b'YES', b'NO', b'NO', b'NO', b'NO'],
       [b'NO', b'NO', b'YES', b'NO', b'NO'],
       [b'NO', b'NO', b'NO', b'NO', b'NO']], dtype=object)>

In [111]:
tf.one_hot(tf7, depth=5, on_value=True, off_value=False)

<tf.Tensor: shape=(7, 5), dtype=bool, numpy=
array([[False, False, False, False,  True],
       [False, False,  True, False, False],
       [False, False, False,  True, False],
       [False,  True, False, False, False],
       [ True, False, False, False, False],
       [False, False,  True, False, False],
       [False, False, False, False, False]])>